## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
wine = datasets.load_wine()

x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size = 0.25, random_state = 87)

#載入模型(預設)
clf = GradientBoostingRegressor()

#訓練模型
clf.fit(x_train, y_train)

#預測結果
y_pred = clf.predict(x_test)

print(metrics.mean_squared_error(y_test, y_pred))

0.009292184080955154


In [4]:
#尋找超參數
n_estimators = [100, 200, 300, 400, 500]
max_depth = [1, 2, 3, 4, 5]
min_samples_leaf = [1, 2, 3, 4, 5]

#彙整超參數數值於字典
param_grid = dict(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf = min_samples_leaf)
param_grid

{'n_estimators': [100, 200, 300, 400, 500],
 'max_depth': [1, 2, 3, 4, 5],
 'min_samples_leaf': [1, 2, 3, 4, 5]}

In [5]:
#建立搜尋物件
grid_search = GridSearchCV(clf, param_grid=param_grid, scoring = "neg_mean_squared_error", n_jobs = -1, verbose = 1)

#開始尋找最佳參數
grid_result = grid_search.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:    5.5s finished


In [6]:
#確認最佳參數
print("Best Accuracy: %f using %s" %(grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.048681 using {'max_depth': 2, 'min_samples_leaf': 5, 'n_estimators': 400}


In [7]:
#使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(n_estimators=grid_result.best_params_["n_estimators"],
                                          max_depth=grid_result.best_params_["max_depth"],
                                          min_samples_leaf=grid_result.best_params_["min_samples_leaf"])

#訓練最佳模型
clf_bestparam.fit(x_train, y_train)

#取得預測值
y_pred = clf_bestparam.predict(x_test)

print(metrics.mean_squared_error(y_test, y_pred))

0.022770580827859703
